In [1]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Package abc is already up-to-date!
       | Downloading package alpino to /root/nltk_data...
       |   Package alpino is already up-to-date!
       | Downloading package averaged_perceptron_tagger to
       |     /root/nltk_data...
       |   Package averaged_perceptron_tagger is already up-to-date!
       | Downloading package averaged_perceptron_tagger_ru to
       |     /root/nltk_data...
       |   Package averaged_perceptron_tagger_ru is already up-to-
       |       date!
       | Downloading package basque_grammars to /root/nltk_data...
       |   Package basque_grammars is already up-to-date!


True

In [2]:
from gensim.models import LdaMulticore,TfidfModel
from gensim.corpora import Dictionary
import pandas as pd 
import re 
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import WordNetLemmatizer

In [3]:
#from google.colab import drive
#drive.mount('/content/drive')

In [4]:
data = pd.read_csv('/content/papers.csv',engine='python', encoding='utf-8', error_bad_lines=False)
data.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
Skipping line 160: unexpected end of data


,id,year,title,event_type,pdf_name,abstract,paper_text
0,1,1987,Self-Organization of Associative Database and ...,NaN,1-self-organization-of-associative-database-an...,Abstract Missing,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...
1,10,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,10-a-mean-field-theory-of-layer-iv-of-visual-c...,Abstract Missing,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...
2,100,1988,Storing Covariance by the Associative Long-Ter...,NaN,100-storing-covariance-by-the-associative-long...,Abstract Missing,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...
3,1000,1994,Bayesian Query Construction for Neural Network...,NaN,1000-bayesian-query-construction-for-neural-ne...,Abstract Missing,Bayesian Query Construction for Neural\nNetwor...
4,1001,1994,"Neural Network Ensembles, Cross Validation, an...",NaN,1001-neural-network-ensembles-cross-validation...,Abstract Missing,"Neural Network Ensembles, Cross\nValidation, a..."


In [5]:
data = data.drop(['year','event_type','pdf_name','abstract'],axis=1)

In [6]:
import re
import string
def text_cleaning(text):
    
    #Converting text into lowercase
    text = str(text).lower()
    
    #Removing square brackets from the text
    text = re.sub('\[.*?\]','',text)
    
    
    #Removing links starting with (https or www)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    
    #Removing <"text"> type of text 
    text = re.sub('<.*?>+','',text)
    
    #Removing punctuations
    text = re.sub("[%s]" % re.escape(string.punctuation),'',text)
    
    #Removing new lines
    text = re.sub("\n",'',text)
    
    #Removing alphanumeric numbers 
    text = re.sub('\w*\d\w*','',text)
    
    return(text)

In [7]:
data['cleaned_text']=data['paper_text'].apply(text_cleaning)
data.head()

,id,title,paper_text,cleaned_text
0,1,Self-Organization of Associative Database and ...,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...,of associative databaseand its applicationshi...
1,10,A Mean Field Theory of Layer IV of Visual Cort...,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...,mean field theory of layer iv of visual corte...
2,100,Storing Covariance by the Associative Long-Ter...,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...,covariance by the associativelongterm potenti...
3,1000,Bayesian Query Construction for Neural Network...,Bayesian Query Construction for Neural\nNetwor...,bayesian query construction for neuralnetwork ...
4,1001,"Neural Network Ensembles, Cross Validation, an...","Neural Network Ensembles, Cross\nValidation, a...",neural network ensembles crossvalidation and a...


In [8]:
len(data)

158

In [9]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

wordnet = WordNetLemmatizer()
def remove_stopwords(text):
    text = text.split()
    text = [wordnet.lemmatize(word) for word in text if word not in set(stopwords.words('english'))]
    text = " ".join(text)
    return(text)

data['cleaned_text']=data['cleaned_text'].apply(remove_stopwords)
data.head()

,id,title,paper_text,cleaned_text
0,1,Self-Organization of Associative Database and ...,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...,associative databaseand applicationshisashi su...
1,10,A Mean Field Theory of Layer IV of Visual Cort...,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...,mean field theory layer iv visual cortexand ap...
2,100,Storing Covariance by the Associative Long-Ter...,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...,covariance associativelongterm potentiation de...
3,1000,Bayesian Query Construction for Neural Network...,Bayesian Query Construction for Neural\nNetwor...,bayesian query construction neuralnetwork mode...
4,1001,"Neural Network Ensembles, Cross Validation, an...","Neural Network Ensembles, Cross\nValidation, a...",neural network ensemble crossvalidation active...


In [22]:
preprocessed_text = data['cleaned_text']
preprocessed_text = [d.split() for d in preprocessed_text]

In [23]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [25]:
dictionary = gensim.corpora.Dictionary(preprocessed_text)

In [26]:
corpus = [dictionary.doc2bow(doc) for doc in preprocessed_text]

In [27]:
tfidf = TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [28]:
lda_model_tfidf = LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

In [29]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print(f'Topic: {idx} Word: {topic}')
    print('\n')

Topic: 0 Word: 0.000*"image" + 0.000*"kernel" + 0.000*"motion" + 0.000*"network" + 0.000*"images" + 0.000*"units" + 0.000*"training" + 0.000*"u" + 0.000*"hidden" + 0.000*"object"


Topic: 1 Word: 0.000*"spike" + 0.000*"kernel" + 0.000*"policy" + 0.000*"image" + 0.000*"units" + 0.000*"neurons" + 0.000*"network" + 0.000*"neuron" + 0.000*"training" + 0.000*"xi"


Topic: 2 Word: 0.000*"network" + 0.000*"neurons" + 0.000*"bound" + 0.000*"x" + 0.000*"kernel" + 0.000*"neuron" + 0.000*"training" + 0.000*"xi" + 0.000*"cells" + 0.000*"hidden"


Topic: 3 Word: 0.000*"policy" + 0.000*"network" + 0.000*"neurons" + 0.000*"clustering" + 0.000*"state" + 0.000*"x" + 0.000*"hidden" + 0.000*"visual" + 0.000*"neuron" + 0.000*"image"


Topic: 4 Word: 0.000*"network" + 0.000*"image" + 0.000*"" + 0.000*"motion" + 0.000*"" + 0.000*"" + 0.000*"" + 0.000*"" + 0.000*"training" + 0.000*""


Topic: 5 Word: 0.000*"" + 0.000*"kernel" + 0.000*"" + 0.000*"" + 0.000*"" + 0.000*"" + 0.000*"state" + 0.000*"ima